In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers datasets folium==0.2.1

     |████████████████████████████████| 5.8 MB 13.0 MB/s 
     |████████████████████████████████| 452 kB 57.8 MB/s 
     |████████████████████████████████| 69 kB 8.9 MB/s 
     |████████████████████████████████| 182 kB 65.6 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 
     |████████████████████████████████| 212 kB 81.6 MB/s 
     |████████████████████████████████| 132 kB 75.2 MB/s 
     |████████████████████████████████| 127 kB 3.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.5 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.


In [ ]:
%cd /content/drive/MyDrive/메인프로젝트(5조)/chat_bot/train/Poly-Encoder

/content/drive/.shortcut-targets-by-id/1XptfKIuBrC2ZUPhu0sfqYQshWVelCsg_/메인프로젝트(5조)/chat_bot/train/Poly-Encoder


스페셜 토큰 리스트화

In [ ]:
import pandas as pd
sp = pd.read_csv('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/special_token/token_filtering_plus_movie.csv')
sp_list = []
for i in range(len(sp['special token'])):
  sp_list.append(sp['special token'][i])

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AutoModel
from encoder import PolyEncoder
from transform import SelectionJoinTransform, SelectionSequentialTransform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

PATH = '/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/train/Poly-Encoder/chatbot_output_plus_mv_qin_sy/poly_16_pytorch_model_32.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(sp_list, special_tokens=True)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=256)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=128)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(PATH))
model.to(device)
model.device

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device(type='cuda', index=0)

함수 설정

In [ ]:
def context_input(context):
    context_input_ids, context_input_masks = context_transform(context)
    contexts_token_ids_list_batch, contexts_input_masks_list_batch = [context_input_ids], [context_input_masks]

    long_tensors = [contexts_token_ids_list_batch, contexts_input_masks_list_batch]

    contexts_token_ids_list_batch, contexts_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return contexts_token_ids_list_batch, contexts_input_masks_list_batch

In [ ]:
def response_input(candidates):
    responses_token_ids_list, responses_input_masks_list = response_transform(candidates)
    responses_token_ids_list_batch, responses_input_masks_list_batch = [responses_token_ids_list], [responses_input_masks_list]

    long_tensors = [responses_token_ids_list_batch, responses_input_masks_list_batch]

    responses_token_ids_list_batch, responses_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return responses_token_ids_list_batch, responses_input_masks_list_batch

In [ ]:
def embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
        
        ctx_out = model.bert(contexts_token_ids_list_batch, contexts_input_masks_list_batch)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(model.poly_m, dtype=torch.long).to(contexts_token_ids_list_batch.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(1, model.poly_m)
        poly_codes = model.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = model.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        return embs

In [ ]:
def cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
                
        batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape # res_cnt is 1 during training
        responses_token_ids_list_batch = responses_token_ids_list_batch.view(-1, seq_length)
        responses_input_masks_list_batch = responses_input_masks_list_batch.view(-1, seq_length)
        cand_emb = model.bert(responses_token_ids_list_batch, responses_input_masks_list_batch)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        return cand_emb

In [ ]:
def loss(embs, cand_emb, contexts_token_ids_list_batch, responses_token_ids_list_batch):
    batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape

    ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, bs, dim]
    # print(ctx_emb)
    ctx_emb = ctx_emb.squeeze()
    # print(ctx_emb)
    dot_product = (ctx_emb*cand_emb) # [bs, bs]
    # print(dot_product)
    dot_product = dot_product.sum(-1)
    print(dot_product)
    mask = torch.eye(batch_size).to(contexts_token_ids_list_batch.device) # [bs, bs]
    print(mask)
    loss = F.log_softmax(dot_product, dim=-1)
    print(loss)
    loss = loss * mask
    print(loss)
    loss = (-loss.sum(dim=1))
    print(loss)
    loss = loss.mean()
    print(loss)
    return loss

In [ ]:
def score(embs, cand_emb):
    with torch.no_grad():
        model.eval()

        ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
        dot_product = (ctx_emb*cand_emb).sum(-1)
        
        return dot_product

### 데이터 검증

In [ ]:
import pickle

with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/trian_pickle/train_data_source_hogwart_plus_mv_qin_sy_32.pickle', 'rb') as f:
    train = pickle.load(f)
with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/trian_pickle/dev_data_source_hogwart_plus_mv_qin_sy_32.pickle', 'rb') as f:
    dev = pickle.load(f)

### 챗봇 테이블 생성

In [ ]:
data = {
    'context' : [],
    'response': []
}

In [ ]:
for i in train:
    data['context'].append(i['context'])
    data['response'].append([i['responses'][0]])

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data)

### generate cand_embs & create tensor table

In [ ]:
response_input_srs = df['response'].apply(response_input)
response_input_lst = response_input_srs.to_list()

cand_embs_lst = []
for i in response_input_lst:
    cand_embs_lst.append(cand_emb_gen(*i).to('cpu'))

In [ ]:
df['response embedding'] = cand_embs_lst
df[['response', 'response embedding']]

,response,response embedding
0,[만 11~18세까지의 청소년 마법사를 교육하는 7학년제 영국의 유일한 마법 학교야.],"[[[tensor(0.6643), tensor(-0.9415), tensor(-0...."
1,"[호그와트는 990년대에 고드릭 그리핀도르, 헬가 후플푸프, 로위너 래번클로, 살라...","[[[tensor(1.1075), tensor(-1.1483), tensor(-0...."
2,[호그와트를 가려면 킹스 크로스역 9와 4분의3 승강장에서 기차를 타고 가야해.],"[[[tensor(0.2449), tensor(-0.9422), tensor(-0...."
3,[허락받지 않은 머글들에게는 보이지 않아. 호그와트 앞을 지나가려고 하면 다 쓰려져...,"[[[tensor(0.8132), tensor(-0.8486), tensor(-0...."
4,[호그와트는 기본적으로 학생의 안전을 매우 중요시하기 때문에 온갖 방어마법이나 고대...,"[[[tensor(0.4714), tensor(-1.0047), tensor(-0...."
...,...,...
6533,[찢어지게 가난해],"[[[tensor(0.8119), tensor(-1.1329), tensor(-0...."
6534,[아니 순혈 마법사야],"[[[tensor(1.3291), tensor(-1.3513), tensor(-0...."
6535,[물푸레나무와 유니콘의 털로 만들어졌어],"[[[tensor(0.7372), tensor(-1.1358), tensor(-0...."
6536,[자신감만 찾으면 충분히 잘해],"[[[tensor(0.5766), tensor(-1.2668), tensor(-0...."


In [ ]:
cand_embs = cand_embs_lst[0]
for idx in range(1, len(cand_embs_lst)):
    y = cand_embs_lst[idx]
    cand_embs = torch.cat((cand_embs, y), 1)

In [ ]:
cand_embs = cand_embs.to(device)

저장

In [ ]:
%cd /content

/content


In [ ]:
with open('best_cand_embs_knows.pickle', 'wb') as fw:
  pickle.dump(cand_embs, fw)

불러오기

In [ ]:
import pickle
with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/train/Poly-Encoder/cand_embs.pickle', 'rb') as f:
    cand_embs = pickle.load(f)
cand_embs.to(device)

tensor([[[ 1.0991e+00, -1.3831e+00, -2.8414e-01,  ..., -6.0775e-01,
           1.1000e-01,  1.9797e-01],
         [ 1.8803e+00, -8.8293e-01, -5.8914e-01,  ..., -5.4705e-01,
          -1.7443e-01,  3.7579e-01],
         [ 8.3584e-01, -8.7785e-01, -5.5983e-01,  ..., -6.9370e-01,
          -5.0204e-01,  1.3792e-01],
         ...,
         [ 1.3138e+00, -1.2395e+00, -5.4559e-01,  ..., -1.9931e-01,
          -6.3562e-01, -3.4643e-01],
         [ 1.3322e+00, -1.4324e+00, -9.9906e-02,  ..., -4.7118e-01,
          -8.4083e-02,  9.2328e-01],
         [ 1.4025e+00, -1.4548e+00,  2.8299e-01,  ..., -5.0450e-01,
           2.0910e-01, -6.4673e-04]]], device='cuda:0')

### Chatbot UI

챗봇 구현 요약 코드

In [ ]:
while True:
    query = [str(input())]
    best_num = -1
    embs = embs_gen(*context_input(query))
    s = score(embs, cand_embs)
    idx = int(s[0].sort()[-1][best_num])
    best_answer = df['response'][idx][0]
    if query == ['끝']:
        break
        
    print(best_answer)

디멘터가 나타났다!
익스펙토 패트로눔!
끝
